<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=dbd79e579880831e135ca148b437f5e38364ba78fd381f310c54c4cc5ace9614
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-10 15:10:05
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.41 C
-------------------
Today PnL: 88.47 K (0.63%)
Current PnL: -17.94 L (-12.07%)
CY Booked + Current PnL: -6.91 L (-4.65%)
-------------------
Total profit:  2.02 L
Total loss:  -19.96 L
-------------------
Total Booked + Current PnL: 20.07 L (16.49%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.81%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.59 L (54.92%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.35%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-12.75,63.0,H-LC,11.21,174427.0,18250.0,7849.0,1.32,11.69,4.50,16.71,22.0,2.33,1.34,31.16,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,56.0,M-LC,4.27,216744.0,15264.0,11943.0,1.12,7.58,5.51,13.51,60.0,1.28,1.66,21.13,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,105.98,63.0,M-SC,5.57,88720.0,-12057.0,12155.0,-0.21,-11.96,13.70,0.09,245.0,-0.99,0.68,16.56,OX40N,NTT,DURABLES
41,ITC,452.00,-37.50,54.0,H-LC,3.26,201957.0,1819.0,19065.0,0.72,0.91,9.44,10.44,4.0,0.10,1.55,6.52,X40,NTT,FMCG
49,NATIONALUM,247.44,-44.49,67.0,H-MC,2.83,110717.0,10213.0,20427.0,1.10,10.16,18.45,30.49,79.0,0.50,0.85,45.86,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.94,61.0,M-MC,7.06,223498.0,-1464.0,169121.0,0.26,-0.65,75.67,74.53,192.0,-0.01,1.72,31.39,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.40,191478.0,472.0,57233.0,-0.04,0.25,29.89,30.21,157.0,0.01,1.47,21.18,X40N,NTT,FINANCE
35,ICICIPRULI,790.00,-20.88,42.0,H-MC,2.00,136063.0,275.0,42479.0,0.65,0.20,31.22,31.48,107.0,0.01,1.04,12.35,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.94,57.0,H-MC,1.76,245638.0,-2093.0,77155.0,-0.79,-0.84,31.41,30.30,92.0,-0.03,1.89,12.33,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.11,44.0,H-MC,3.49,137277.0,1144.0,29981.0,1.10,0.84,21.84,22.86,91.0,0.04,1.05,13.56,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303688.0,-12154.0,124816.0,0.32,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.88,42.0,H-LC,13.75,217000.0,-406.0,53903.0,0.67,-0.19,24.84,24.61,48.0,-0.01,1.67,54.53,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.51,44.0,H-LC,8.27,171930.0,-9829.0,45080.0,-0.15,-5.41,26.22,19.39,86.0,-0.22,1.32,2.30,X40N,NTT,BREWERIES
56,RELIANCE,1533.00,-14.04,46.0,H-LC,6.49,214765.0,4399.0,24376.0,0.04,2.09,11.35,13.68,37.0,0.18,1.65,18.96,XY25,NTT,REFINERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.16,230005.0,-11919.0,75189.0,2.66,-4.93,32.69,26.15,8.0,-0.16,1.77,8.69,X40,ATH,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.76,59.0,H-LC,1.33,238950.0,-7201.0,86404.0,2.40,-2.93,36.16,32.18,16.0,-0.08,1.83,32.06,X200,ATH,IT
50,NESTLEIND,1377.00,-7.46,62.0,H-LC,2.35,284848.0,19422.0,37372.0,1.26,7.32,13.12,21.40,11.0,0.52,2.19,14.45,XY25,NTT,FMCG
41,ITC,452.00,-37.50,54.0,H-LC,3.26,201957.0,1819.0,19065.0,0.72,0.91,9.44,10.44,4.0,0.10,1.55,6.52,X40,NTT,FMCG
20,CIPLA,1795.00,-19.78,53.0,H-LC,5.95,214213.0,9713.0,31704.0,0.77,4.75,14.80,20.25,10.0,0.31,1.64,13.90,X40N,BTT,PHARMA
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303688.0,-12154.0,124816.0,0.32,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.00,-17.83,61.0,H-MC,8.19,81830.0,-20822.0,46422.0,5.25,-20.28,56.73,24.94,98.0,-0.45,0.63,24.39,OX40N,NTT,IT
76,TRIDENT,48.00,-6.93,59.0,M-SC,5.98,72470.0,-19784.0,46091.0,4.38,-21.45,63.60,28.51,224.0,-0.43,0.56,24.01,XR,NTT,TEXTILES
12,BANKINDIA,190.00,-30.43,62.0,H-MC,5.52,180515.0,707.0,111703.0,4.08,0.39,61.88,62.52,88.0,0.01,1.39,34.61,XR,NTT,BANKS
79,UNIONBANK,163.00,-16.28,53.0,M-LC,8.96,150573.0,9733.0,34752.0,4.08,6.91,23.08,31.59,66.0,0.28,1.16,35.23,XY24,NTT,BANKS
54,RAJOOENG,143.33,-34.52,39.0,H-SC,10.70,99170.0,-3330.0,44160.0,3.66,-3.25,44.53,39.83,114.0,-0.08,0.76,3.66,AR,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.00,-329.57,58.0,H-SC,8.68,97500.0,-35786.0,120383.0,-3.77,-26.85,123.47,63.47,139.0,-0.30,0.75,63.43,XR,NTT,CHEMICALS
8,ATULAUTO,844.00,3640.54,60.0,M-SC,7.64,116572.0,-30796.0,82615.0,-2.11,-20.90,70.87,35.16,236.0,-0.37,0.90,19.07,XY24,NTT,AUTO
46,LAOPALA,464.00,116.69,52.0,H-SC,3.07,68769.0,-32077.0,57903.0,-2.01,-31.81,84.20,25.61,142.0,-0.55,0.53,27.50,AR,NTT,CERAMICS
65,SIEMENS,7969.85,-2.40,51.0,H-LC,29.43,156945.0,-29150.0,241554.0,-1.60,-15.66,153.91,114.13,15.0,-0.12,1.21,15.58,AR,ATH,ELECTRICAL
44,KANSAINER,340.00,-67.97,55.0,H-SC,2.47,220320.0,-49347.0,85682.0,-1.56,-18.30,38.89,13.47,138.0,-0.58,1.69,11.43,XY24,NTT,PAINTS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,71.0,H-SC,12.82,130437.0,-12042.0,31566.0,1.77,-8.45,24.20,13.70,163.0,-0.38,1.00,53.95,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,105.98,63.0,M-SC,5.57,88720.0,-12057.0,12155.0,-0.21,-11.96,13.70,0.09,245.0,-0.99,0.68,16.56,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,55.0,H-SC,2.47,220320.0,-49347.0,85682.0,-1.56,-18.30,38.89,13.47,138.0,-0.58,1.69,11.43,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,113.73,48.0,M-SC,3.03,140120.0,-34439.0,34470.0,0.20,-19.73,24.60,0.02,221.0,-1.00,1.08,21.88,OX40N,NTT,PAINTS
72,TATAELXSI,9161.0,-17.83,61.0,H-MC,8.19,81830.0,-20822.0,46422.0,5.25,-20.28,56.73,24.94,98.0,-0.45,0.63,24.39,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.76,52.0,H-LC,13.94,292237.0,-53719.0,177563.0,1.95,-15.53,60.76,35.80,1.0,-0.30,2.24,3.53,X40,BTT,IT
39,INFY,2275.00,-16.37,58.0,H-LC,10.11,326231.0,13343.0,158353.0,1.81,4.26,48.54,54.87,3.0,0.08,2.50,11.20,X40,BTT,IT
41,ITC,452.00,-37.50,54.0,H-LC,3.26,201957.0,1819.0,19065.0,0.72,0.91,9.44,10.44,4.0,0.10,1.55,6.52,X40,NTT,FMCG
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303688.0,-12154.0,124816.0,0.32,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG
1,ABB,7934.00,-40.74,48.0,H-LC,12.54,247589.0,-14030.0,133252.0,0.51,-5.36,53.82,45.57,7.0,-0.11,1.90,5.35,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.63,53.0,H-MC,3.54,189110.0,24440.0,43533.0,0.22,14.84,23.02,41.28,89.0,0.56,1.45,24.01,X40N,ATH,AC
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.40,191478.0,472.0,57233.0,-0.04,0.25,29.89,30.21,157.0,0.01,1.47,21.18,X40N,NTT,FINANCE
84,VOLTAS,1918.49,-0.38,54.0,H-MC,3.32,208905.0,17163.0,78862.0,-1.39,8.95,37.75,50.08,99.0,0.22,1.60,16.36,XY25,ATH,AC
24,DABUR,735.00,-2.03,63.0,H-MC,5.59,213330.0,16302.0,73322.0,0.51,8.27,34.37,45.49,102.0,0.22,1.64,22.39,XY24,BTT,FMCG
18,CAMS,5211.76,-4.05,54.0,H-SC,4.92,109001.0,6997.0,36930.0,1.05,6.86,33.88,43.06,122.0,0.19,0.84,25.15,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-44.49,67.0,H-MC,2.83,110717.0,10213.0,20427.0,1.10,10.16,18.45,30.49,79.0,0.50,0.85,45.86,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.15,64.0,M-SC,2.47,102885.0,11386.0,67369.0,0.04,12.44,65.48,86.07,223.0,0.17,0.79,49.41,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.09,49.0,H-SC,12.14,136475.0,13139.0,120630.0,0.01,10.65,88.39,108.45,119.0,0.11,1.05,35.39,AR,ATH,MISC
86,WIPRO,420.00,-9.71,60.0,M-LC,6.48,158639.0,7694.0,101767.0,2.70,5.10,64.15,72.51,53.0,0.08,1.22,11.43,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,62.0,H-MC,5.52,180515.0,707.0,111703.0,4.08,0.39,61.88,62.52,88.0,0.01,1.39,34.61,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6750.00,53.0,L-SC,6.09,75187.0,-18623.0,95239.0,0.63,-19.85,126.67,81.67,269.0,-0.20,0.58,45.60,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-4.34,46.0,H-SC,14.69,89023.0,-11940.0,106472.0,2.00,-11.83,119.60,93.63,148.0,-0.11,0.68,32.60,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,59.0,L-SC,42.14,89818.0,-23731.0,63780.0,0.19,-20.90,71.01,35.27,268.0,-0.37,0.69,119.83,XR,NTT,HOTELS
48,MASFIN,398.61,-16.44,57.0,H-SC,13.01,94830.0,-3150.0,24751.0,-0.94,-3.21,26.10,22.05,152.0,-0.13,0.73,37.84,XR,ATH,FINANCE
82,VALIANTORG,838.00,-329.57,58.0,H-SC,8.68,97500.0,-35786.0,120383.0,-3.77,-26.85,123.47,63.47,139.0,-0.30,0.75,63.43,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,59.0,L-SC,42.14,89818.0,-23731.0,63780.0,0.19,-20.90,71.01,35.27,268.0,-0.37,0.69,119.83,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,63.0,M-SC,3.96,81918.0,-20292.0,113686.0,0.15,-19.85,138.78,91.37,208.0,-0.18,0.63,32.98,XY25,NTT,FINANCE
75,TITAGARH,1548.0,-3.20,64.0,H-SC,7.12,158568.0,-33447.0,112330.0,2.51,-17.42,70.84,41.08,158.0,-0.30,1.22,34.37,XY24,NTT,ENGINEERING
8,ATULAUTO,844.0,3640.54,60.0,M-SC,7.64,116572.0,-30796.0,82615.0,-2.11,-20.90,70.87,35.16,236.0,-0.37,0.90,19.07,XY24,NTT,AUTO
64,SHALBY,327.0,1086.60,75.0,M-SC,2.33,161414.0,-19729.0,65195.0,0.67,-10.89,40.39,25.10,235.0,-0.30,1.24,29.36,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.35
1,25,44.42
2,50,75.09


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.88
LC    35.43
MC    23.72
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.67
X40      16.08
X40N     13.13
XY25     10.28
XR        9.57
AR        8.35
OX40N     7.78
X200      1.83
MH        1.78
X5K       1.44
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.86
H-SC    25.01
H-MC    20.64
M-SC    14.19
M-LC     7.46
M-MC     2.75
L-SC     1.68
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.00,-15.07,73.33
FMCG,12.30,-3.59,38.70
FINANCE,9.07,-18.38,60.95
BANKS,7.79,-12.39,60.12
PAINTS,6.16,-15.25,41.05
MISC,6.06,-16.58,83.41
ELECTRICAL,5.84,-8.45,68.23
AC,3.80,2.30,30.79
AUTO,3.48,-16.00,68.32


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2732631.0,22
AR,1183569.0,9
XR,1049774.0,13
X40,872133.0,10
X40N,565933.0,10
OX40N,446806.0,10
XY25,411676.0,7
SR,197156.0,2
X5K,129537.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2768322.0,28
M-SC,1398900.0,17
H-LC,1366798.0,15
H-MC,1155852.0,15
M-LC,407289.0,5
M-MC,319548.0,2
L-SC,243095.0,3
L-MC,59827.0,1
L-LC,39401.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,915814.0,6
M-SC,XY24,816329.0,7
H-SC,AR,649061.0,5
H-LC,X40,580488.0,5
H-SC,XR,503998.0,6
H-MC,XY24,462995.0,4
H-LC,AR,374806.0,2
M-MC,XY24,319548.0,2
H-SC,X40N,247546.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
